# 大会記録のデータの移行
### postgresqlへの接続

In [ ]:
import psycopg2
from psycopg2.errors import DuplicateColumn

conn = psycopg2.connect('user=honomara dbname=honomara password=honomara')

def get_data(sql):
    with conn.cursor() as cur:
        cur.execute(sql)
        data = cur.fetchall()
    return data

def get_data_dict(sql):
    with conn.cursor() as cur:
        cur.execute(sql)
        data = cur.fetchall()
        col_name = [col.name for col in cur.description]
    return [ dict(zip(col_name,d)) for d in data]


def get_colname(table):
    with conn.cursor() as cur:
        cur.execute('SELECT * FROM {} LIMIT 1;'.format(table))
        data = [col.name for col in cur.description]
    return data

def get_table_list():
    return [table[0] for table in get_data('SELECT relname FROM pg_stat_user_tables;')]

def get_sequence_list():
    return [seq[0] for seq in get_data('''SELECT c.relname FROM pg_class c LEFT join pg_user u ON c.relowner = u.usesysid WHERE c.relkind = 'S';''')]

def exec_transactions(sqls):
    with conn.cursor() as cur:
        for sql in sqls:
            cur.execute(sql)
    conn.commit()

### mysqlへの接続

In [ ]:
import mysql.connector
conn2 = mysql.connector.connect(
    host = 'localhost',
    port = 3306,
    user = 'honomara',
    password =  'honomara',
    database = 'honomara',
)

cur2 = conn2.cursor(dictionary=True)


### race_base, race, race_type, resultテーブルの追加

In [ ]:
create_race_base_table = '''
CREATE TABLE IF NOT EXISTS race_base (
    `race_name`      VARCHAR(60) PRIMARY KEY,
    `race_name_kana` VARCHAR(60),
    `prefecture`     VARCHAR(30),
    `comment`        TEXT
);
'''

create_race_table = '''
CREATE TABLE IF NOT EXISTS race (
    `id`        INT PRIMARY KEY AUTO_INCREMENT,
    `race_name` VARCHAR(60) NOT NULL,
    `date`      DATE NOT NULL,
    `comment`   TEXT
);
'''
create_race_type_table = '''
CREATE TABLE IF NOT EXISTS race_type (
    `id`         INT PRIMARY KEY AUTO_INCREMENT,
    `race_type`  VARCHAR(30) NOT NULL DEFAULT 'road', 
    `show_name`  VARCHAR(30), 
    `ranking`    INT NOT NULL DEFAULT 100, 
    `duration`   FLOAT,
    `distance`   FLOAT,
    `comment`    TEXT
);
'''

create_result_table = '''
CREATE TABLE IF NOT EXISTS result (
    `member_id`     INT NOT NULL, 
    `race_type_id`  INT NOT NULL, 
    `race_id`       INT NOT NULL, 
    `result`        INT NOT NULL,
    `comment`       TEXT
);
'''

cur2.execute(create_race_base_table)
cur2.execute(create_race_table)
cur2.execute(create_race_type_table)
cur2.execute(create_result_table)
conn2.commit()
#cur2.execute('''ALTER TABLE race ADD CONSTRAINT FOREIGN KEY (race_name) REFERENCES race_base(race_name);''')
cur2.execute('''ALTER TABLE result ADD CONSTRAINT FOREIGN KEY (member_id)    REFERENCES member(id);''')
cur2.execute('''ALTER TABLE result ADD CONSTRAINT FOREIGN KEY (race_type_id) REFERENCES race_type(id);''')
cur2.execute('''ALTER TABLE result ADD CONSTRAINT FOREIGN KEY (race_id)      REFERENCES race(id);''')
conn2.commit()

### race_base テーブルの移行


In [ ]:
import re
ds = get_data('SELECT DISTINCT race_name FROM race;')
ds = [ (d[0].strip()) for d in ds]#  if re.match('横浜',d[0])]
ds = set(ds)
ds = list(ds)
ds.sort()
prev = ''
# for d in ds:
#     if prev[:5:] == d[:5:]:
#         print(d,prev)
#     prev = d
    
    
table= {}
table["いたばしリバーサイド・ハーフマラソン"] = "いたばしリバーサイドハーフマラソン"
table["おきなわマラソン_"] = "おきなわマラソン"
table["戸田・彩湖フルマラソン&ウルトラマラソン"] = "戸田・彩湖フルマラソン＆ウルトラマラソン"
ds = [table.get(d,d) for d in ds]
ds = set(ds)
ds = list(ds)
ds.sort()

for d in ds:
    cur2.execute('INSERT INTO race_base (race_name) VALUES (%s);',(d,))
    
conn2.commit()

### raceテーブルの移行

In [ ]:
table= {}
table["いたばしリバーサイド・ハーフマラソン"] = "いたばしリバーサイドハーフマラソン"
table["おきなわマラソン_"] = "おきなわマラソン"
table["戸田・彩湖フルマラソン&ウルトラマラソン"] = "戸田・彩湖フルマラソン＆ウルトラマラソン"

n =get_data('SELECT count(*) FROM race; ')[0][0] 
ofst = 0
limit = n
while ofst < n:
    ds = get_data_dict('SELECT race_id,race_name,date FROM race ORDER BY race_id OFFSET {ofst} LIMIT {lmt};'.format(ofst=ofst,lmt=limit))
    for d in ds:
        d['race_name'] = table.get(d['race_name'],d['race_name']).strip()
#         print(d)
        cur2.execute('INSERT INTO race (id, race_name, date) VALUES (%s,%s,%s);',(d['race_id'],d['race_name'],d['date']))
    ofst += limit
conn2.commit()

### race_type テーブルの移行(旧distance)

In [ ]:
from re import search
ds = get_data_dict('''
SELECT distance_id AS race_type_id, distance AS distance, distance_name AS show_name, ranking  FROM distance ORDER BY ranking;
''')
time = list(filter(lambda d: search('時間', d['show_name']) ,ds))
road = list(filter(lambda d: search('マラソン|キロ|マイル', d['show_name']) ,ds))
track = list(filter(lambda d: search('メートル', d['show_name']) ,ds))

# print(len(ds))
# print(len(time))
# print(len(road))
# print(len(track))

for t in time:
    t['duration'] =  float(t['show_name'].replace('時間走','')) * 3600
    t['distance'] =  None
    t['race_type'] = 'time'
    print(t)
    cur2.execute('''INSERT INTO race_type (id, race_type, duration, show_name, ranking) VALUES (%s,%s,%s,%s,%s);'''
                 ,(t['race_type_id'],t['race_type'],t['duration'],t['show_name'],t['ranking']))


    
for t in track:
    t['race_type'] = 'track'
    t['distance'] =  float(t['distance'])
    print(t)
    cur2.execute('''INSERT INTO race_type (id, race_type, duration, show_name, ranking) VALUES (%s,%s,%s,%s,%s);'''
                 ,(t['race_type_id'],t['race_type'],t['distance'],t['show_name'],t['ranking']))

    
for t in road:
    t['distance'] =  float(t['distance'])
    t['race_type'] = 'road'
    print(t)
    cur2.execute('''INSERT INTO race_type (id, race_type, duration, show_name, ranking) VALUES (%s,%s,%s,%s,%s);'''
                 ,(t['race_type_id'],t['race_type'],t['distance'],t['show_name'],t['ranking']))
conn2.commit()

### results テーブルの移行

In [ ]:
n =get_data('SELECT count(*) FROM result; ')[0][0] 
print(n)
ofst = 0
limit = 10
while ofst < n:
    ds = get_data_dict('''
    SELECT person_id as member_id, race_id, distance_id as race_type_id,time as result, display_time, comment FROM result
    ORDER BY race_id OFFSET {ofst} LIMIT {lmt};
    '''.format(ofst=ofst,lmt=limit))
    for d in ds:
        if not d['result']:
            continue
        print(d)
        cur2.execute('''INSERT INTO result (member_id, race_id, race_type_id, result, comment) VALUES (%s,%s,%s,%s,%s);'''
                     ,(d['member_id']
                       ,d['race_id']
                       ,d['race_type_id']
                       , int(d['result'].total_seconds())
                       ,'display_time:{} {}'.format(d['display_time'],'' if d['comment'] is None else d['comment'])))
        pass
    
    ofst += limit
conn2.commit()

## miscellaneous info

In [ ]:
d = get_data('SELECT DISTINCT race_name,count(*) as count FROM race GROUP BY race_name ORDER BY race_name;')
len(d), d

In [ ]:
get_colname('race')

In [ ]:
get_data('SELECT * FROM result LIMIT 10;')

In [ ]:
get_colname('result')

In [ ]:
get_colname('distance')

In [ ]:
get_colname('person')

In [ ]:
get_data('SELECT race_name, count(*) AS cnt FROM race GROUP BY race_name ORDER BY cnt DESC ;')

In [ ]:
get_data('''
SELECT race_name, year ,count(*) AS cnt FROM race
GROUP BY race_name, year 
ORDER BY cnt DESC''')

In [ ]:
get_data('''SELECT person.name, race.race_name, race.year, distance.distance_name, display_time, time, comment AS cnt 
    FROM result 
    JOIN person ON result.person_id = person.person_id
    JOIN race ON result.race_id = race.race_id
    JOIN distance ON result.distance_id = distance.distance_id
    WHERE (comment IS NOT NULL) AND (comment != '')
    LIMIT 100 ''')

In [ ]:
cur2.execute('''SELECT result.result, result.comment, race.race_name, race.date, race_type.show_name, member.show_name FROM result 
JOIN race ON result.race_id=race.id
JOIN member ON result.member_id = member.id
JOIN race_type ON result.race_type_id=race_type.id
 ORDER BY race.date DESC LIMIT 200;''')
cur2.fetchall()